In [1]:
import os
os.environ["WANDB_DISABLED"]="true"

# Install required packages

In [2]:
! pip install -q datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] evaluate sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not 

In [3]:
import transformers

In [4]:
import evaluate
metric = evaluate.load("sacrebleu")

2025-04-23 15:32:12.794396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745422333.018980      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745422333.081567      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict

# 1. Đọc file CSV
df = pd.read_csv("/kaggle/input/mt-dataset/dataset.csv")

# 2. Làm sạch và chọn cột
df = df[["en", "vi"]]
df = df.dropna()

# 3. Giới hạn còn 10.0000 mẫu
df = df.sample(n=100000, random_state=42).reset_index(drop=True)  # random_state để tái lập kết quả

# 4. Tạo cột 'translation'
df["translation"] = df.apply(lambda row: {"en": row["en"], "vi": row["vi"]}, axis=1)

# 5. Tạo Dataset từ pandas DataFrame
full_dataset = Dataset.from_pandas(df[["translation"]], preserve_index=False)

# 6. Chia train/val/test (80/10/10)
split_dataset = full_dataset.train_test_split(test_size=0.2, seed=42)
val_test = split_dataset["test"].train_test_split(test_size=0.5, seed=42)

# 7. Gộp lại thành DatasetDict
raw_datasets = DatasetDict({
    "train": split_dataset["train"],
    "validation": val_test["train"],
    "test": val_test["test"]
})


In [6]:
# ✅ Kiểm tra thử
print(raw_datasets["train"][1])

{'translation': {'en': 'According to historian David Hill, "Europeans knew little about the geography of the globe" and to "convicts in England, transportation to Botany Bay was a frightening prospect".', 'vi': 'Theo sử gia David Hill, "Người châu Âu biết ít về địa lý toàn cầu" và với "các tù nhân tại Anh, đầy đến vịnh Botany là một viễn cảnh khủng khiếp."'}}


In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-vi"

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

In [9]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "vi"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
preprocess_function(raw_datasets['train'][:2])

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[14289, 2854, 11, 29, 12117, 22, 7, 2946, 1095, 2059, 126, 145, 23482, 2100, 4, 8739, 4166, 29, 8553, 1128, 4745, 73, 488, 14234, 344, 15, 14718, 25132, 4, 2087, 7, 18983, 13, 15, 25376, 1735, 5277, 5722, 5346, 135, 7, 898, 13, 7, 2059, 2, 0], [3629, 10, 11568, 1040, 7711, 4, 117, 37762, 11, 1147, 476, 138, 7, 29897, 13, 7, 10187, 431, 19, 10, 117, 2904, 17155, 29, 11, 22, 5354, 4, 14718, 10, 3875, 29, 17611, 5190, 64, 15, 17741, 7177, 431, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[553, 91, 36, 166, 730, 124, 70, 1130, 2946, 80, 27, 1009, 2443, 3204, 1153, 4, 8739, 4166, 29, 8553, 1128, 4745, 73, 122, 594, 1455, 166, 27, 563, 427, 895, 512, 4, 264, 91, 595, 166, 27, 3653, 412, 4034, 852, 249, 5722, 5346, 124, 

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command

In [11]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
tokenized_datasets.save_to_disk("tokenized_en_vi")

from datasets import load_from_disk
tokenized_datasets = load_from_disk("tokenized_en_vi")

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/80000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [13]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    fp16=True,  
    predict_with_generate=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [17]:
# trainer.train()

In [18]:
import os
for dirname, _, filenames in os.walk('opus-mt-en-vi-finetuned-en-to-vi'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Test

In [19]:
from transformers import MarianMTModel, MarianTokenizer
src_text = ['My name is Sarah and I live in London']

model_name = '/kaggle/input/weight-pretrained-transformer/opus-mt-en-vi-finetuned-en-to-vi/checkpoint-25000'
tokenizer = MarianTokenizer.from_pretrained(model_name)
print(tokenizer.supported_language_codes)

model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

['>>vie<<']


model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

['Tên tôi là Sarah và tôi sống ở London']

# Evaluate


In [20]:
!pip install -q rouge_score
from transformers import MarianMTModel, MarianTokenizer
import evaluate
import torch
from tqdm import tqdm

def load_translation_model(model_path):
    """Tải model và tokenizer"""
    tokenizer = MarianTokenizer.from_pretrained(model_path)
    model = MarianMTModel.from_pretrained(model_path)
    return tokenizer, model

def predict(texts, tokenizer, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """Dịch danh sách câu từ Anh sang Việt"""
    model = model.to(device)
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

def evaluate_model(test_dataset, tokenizer, model, batch_size=8):
    """Tính BLEU và ROUGE scores cho tập test"""
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")
    
    references = []
    predictions = []
    
    # Lấy toàn bộ dữ liệu test
    test_data = test_dataset['translation']
    
    for i in tqdm(range(0, len(test_data), batch_size)):
        batch = test_data[i:i+batch_size]
        
        # Chuẩn bị batch
        src_texts = [item['en'] for item in batch]
        ref_texts = [item['vi'] for item in batch]
        
        # Dịch
        pred_texts = predict(src_texts, tokenizer, model)
        
        # Lưu kết quả
        references.extend([[ref] for ref in ref_texts])  # BLEU cần list of references
        predictions.extend(pred_texts)
    
    # Tính metrics
    bleu_results = bleu_metric.compute(predictions=predictions, references=references)
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)
    
    return {
        'bleu': bleu_results['bleu'],
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL']
    }

results = evaluate_model(raw_datasets["test"], tokenizer, model)

print("\nKết quả đánh giá:")
print(f"BLEU score: {results['bleu']:.4f}")
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")

  Preparing metadata (setup.py) ... done


100%|██████████| 1250/1250 [1:50:36<00:00,  5.31s/it]



Kết quả đánh giá:
BLEU score: 0.3826
ROUGE-1: 0.7583
ROUGE-2: 0.5725
ROUGE-L: 0.6917
